In [ ]:
# import libraries

import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# load FashionMNIST
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# CNN architecture
class CNN_FashionMNIST(nn.Module):
    def __init__(self):
        super(CNN_FashionMNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(64 * 7 * 7, 256)  # after 2 poolings (28→14→7)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# create model
model = CNN_FashionMNIST().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# training
for epoch in range(20):  # increase epochs
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/20, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {acc:.2f}%")

# testing
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Epoch 1/20, Loss: 0.5882, Accuracy: 79.88%
Epoch 2/20, Loss: 0.3530, Accuracy: 87.37%
Epoch 3/20, Loss: 0.3077, Accuracy: 88.83%
Epoch 4/20, Loss: 0.2795, Accuracy: 89.96%
Epoch 5/20, Loss: 0.2597, Accuracy: 90.53%
Epoch 6/20, Loss: 0.2431, Accuracy: 91.21%
Epoch 7/20, Loss: 0.2285, Accuracy: 91.69%
Epoch 8/20, Loss: 0.2161, Accuracy: 92.17%
Epoch 9/20, Loss: 0.2067, Accuracy: 92.39%
Epoch 10/20, Loss: 0.1980, Accuracy: 92.70%
Epoch 11/20, Loss: 0.1860, Accuracy: 93.15%
Epoch 12/20, Loss: 0.1773, Accuracy: 93.46%
Epoch 13/20, Loss: 0.1705, Accuracy: 93.71%
Epoch 14/20, Loss: 0.1639, Accuracy: 93.99%
Epoch 15/20, Loss: 0.1559, Accuracy: 94.14%
Epoch 16/20, Loss: 0.1494, Accuracy: 94.46%
Epoch 17/20, Loss: 0.1418, Accuracy: 94.74%
Epoch 18/20, Loss: 0.1374, Accuracy: 94.88%
Epoch 19/20, Loss: 0.1297, Accuracy: 95.17%
Epoch 20/20, Loss: 0.1247, Accuracy: 95.30%
Test Accuracy: 91.96%
